In [1]:
import os
import copy
import time
import pickle
import sys

import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sian.utils import gettimestamp
from sian.data import Final_TabularDataset
from sian.models import TrainingArgs

from sian.fis import layerwise_FIS_Hyperparameters, batchwise_FIS_Hyperparameters
from sian.interpret import unmasked_FID_Hyperparameters, masked_FID_Hyperparameters
from sian import initalize_the_explainer

from sian import train_mlp_final, do_the_fis_final, train_sian_final #steps 1, 2, and 3
from sian.interpret import plot_all_GAM_functions #step 4


%load_ext autoreload
%autoreload 2

In [2]:
BS = 32
# EP = 100
EP = 10
LR = 5e-3

if True:
    dataset_str = "UCI_275_bike_sharing_dataset"
    preproc_owner = "SIAN2022"
if False:
    dataset_str = "UCI_186_wine_quality"
    preproc_owner = "SIAN2022"
if False:
    dataset_str = "UCI_2_adults_dataset"
    preproc_owner = "InstaSHAP2025"
if False:
    dataset_str = "UCI_31_tree_cover_type_dataset"
    preproc_owner = "InstaSHAP2025"


data_base_path = "../data/"
load_dataset_path = data_base_path
save_dataset_path = data_base_path+dataset_str+"/"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mlp_training_args = TrainingArgs(BS, EP, LR, device)
sian_training_args = TrainingArgs(BS, EP, LR, device)




is_masked_mlp = True;  is_masked_sian = True;
# is_masked_mlp = False; is_masked_sian = False;

FIS_style = 'batchwise'
# FIS_style = 'layerwise'    

MAX_K = None
MAX_K = 3
MAX_K = 2 


In [3]:

results_path = "results/"
exp_datetimestr = gettimestamp()
exp_folder = results_path+exp_datetimestr +'_'+ "demo" +'_simple_testing/'
if not os.path.exists(exp_folder):
    os.makedirs(exp_folder)
print(exp_folder)

results/20250420_010754_demo_simple_testing/


In [4]:

if True: #DEFAULT MODEL PARAMETRIZATION
    mlp_training_args.model_config.net_name = "MLP"
    mlp_training_args.model_config.sizes = [-1, 256, 128, 64, -1]
    mlp_training_args.model_config.is_masked = is_masked_mlp
    mlp_training_args.saving_settings.exp_folder = exp_folder
    
    sian_training_args.model_config.net_name = "SIAN-K"
    sian_training_args.model_config.sizes = [-1, 256, 128, 64, -1]
    sian_training_args.model_config.small_sizes = [-1, 32, 24, 16, -1]
    sian_training_args.model_config.is_masked = is_masked_sian
    sian_training_args.saving_settings.exp_folder = exp_folder

In [5]:
dataset_obj = \
    Final_TabularDataset(dataset_str, preproc_owner=preproc_owner,
                       load_dataset_path=load_dataset_path, 
                       save_dataset_path=save_dataset_path)     


header_dict {'Preprocessed Datetime': '20250413_234719', 'dataset_id': 'UCI_275_bike_sharing_dataset', 'preproc_owner': 'SIAN2022', 'load_dataset_path': '../../data/', 'save_dataset_path': '../../data/UCI_275_bike_sharing_dataset/', 'is_test_split_shuffled': False, 'shuffle_test_split_seed': None, 'trainval_portion': 0.8}
LOADING FROM EXISTING


In [6]:
D = dataset_obj.get_D()
readable_labels = dataset_obj.get_readable_labels()
print(readable_labels)

{0: 'day', 1: 'season', 2: 'year', 3: 'month', 4: 'hour', 5: 'holiday', 6: 'day of week', 7: 'workday', 8: 'weather', 9: 'temperature', 10: 'feels_like_temp', 11: 'humidity', 12: 'wind speed'}


# SIAN Step 1: Train Masked MLP

In [7]:
mlp_results = train_mlp_final(dataset_obj, mlp_training_args)
trained_mlp = mlp_results["trained_mlp"]
val_tensor = mlp_results["val_tensor"]


trnval_shuffle_seed 0
self.trnval_shuffle_seed 0
trnX trnY (9732, 13) (9732, 1)
Epoch 0
MSE for train and val: 1.328242992661506, 1.3235039915965843
--- 2.063 seconds in epoch ---
Epoch 1
MSE for train and val: 0.9837330114463784, 0.9567840402124934
--- 1.493 seconds in epoch ---
Epoch 2
MSE for train and val: 0.8691166635347055, 0.8584403513018878
--- 1.535 seconds in epoch ---
Epoch 3
MSE for train and val: 0.8789926005382099, 0.8597926930910728
--- 1.458 seconds in epoch ---
Epoch 4
MSE for train and val: 0.7084218955216931, 0.6890411729514189
--- 1.500 seconds in epoch ---
Epoch 5
MSE for train and val: 0.7919441803696304, 0.7747316053222189
--- 1.472 seconds in epoch ---
Epoch 6
MSE for train and val: 0.6444322991460378, 0.6339700747383478
--- 1.542 seconds in epoch ---
Epoch 7
MSE for train and val: 0.5735495339207528, 0.5630253633067952
--- 1.556 seconds in epoch ---
Epoch 8
MSE for train and val: 0.5324444254649296, 0.5416093405485685
--- 1.813 seconds in epoch ---
Epoch 9
MSE 

# SIAN Step 2: Masked Archipelago FIS

### setup FID hypers

In [8]:

output_type = "regression"  #TODO: can set to classification when masking version has support (not sobol version)
grouped_features_dict = dataset_obj.get_grouped_feature_dict()
if is_masked_mlp: 
    fid_masking_style = "masking_based"
    score_type_name = "new_arch_inter_sobol_score"
    inc_rem_pel_list = ['inc_inter_sobol_score', 'rem_inter_sobol_score', 'new_arch_inter_sobol_score',] #NOTE: only for batchwise plots
    fis_valX = val_tensor

    my_FID_hypers = masked_FID_Hyperparameters(fid_masking_style, output_type, score_type_name, inc_rem_pel_list,
                                               grouped_features_dict)
else:    
    fid_masking_style = "triangle_marginal"
    score_type_name = "old_arch_inter_score"
    inc_rem_pel_list = ['inc_inter_score', 'rem_inter_score', 'old_arch_inter_score',] #NOTE: only for batchwise plots
    fis_valX = val_tensor.detach().cpu().numpy()
    
    my_FID_hypers = unmasked_FID_Hyperparameters(fid_masking_style, output_type, score_type_name, inc_rem_pel_list,
                                               device, grouped_features_dict)


### setup FIS hypers

In [9]:

if FIS_style=="batchwise":
    max_number_of_rounds = 5
    inters_per_round = 1
    tau_tup=(1.0,0.5,0.33)
    
    tau_thresholds = {}
    for k in range(MAX_K): #NOTE: no good MAX_K = None support yet
        tau_thresholds[k+1] = tau_tup[k]
    
    my_FIS_hypers = batchwise_FIS_Hyperparameters(MAX_K, tau_thresholds, max_number_of_rounds, inters_per_round,
                   # jam_arch, 
                   None, 
                   tuples_initialization=None,pick_underlings=False,fill_underlings=False,PLOTTING=True)

elif FIS_style=="layerwise":

    theta_percentile_mode=True
    theta_tup=(0.8,0.4,0.2)
    tau_tup=(1.0,0.5,0.33)
    
    tau_thresholds, theta_thresholds = {}, {}
    for k in range(MAX_K):
        tau_thresholds[k+1] = tau_tup[k]
        theta_thresholds[k+1] = theta_tup[k]

    my_FIS_hypers = layerwise_FIS_Hyperparameters(MAX_K, tau_thresholds, theta_thresholds, 
                   # jam_arch, 
                   None, 
                   theta_percentile_mode=theta_percentile_mode)
else:
    raise Exception(f"FIS_style={FIS_style} not recognized")


### finalize the FID and FIS hypers

In [10]:
jam_arch = initalize_the_explainer(trained_mlp, my_FID_hypers)
my_FIS_hypers.add_the_explainer(jam_arch)

### run the actual FIS

In [11]:
FIS_algorithm_start_time = time.time()
FIS_interactions = do_the_fis_final(my_FIS_hypers, fis_valX, AGG_K=100)
FIS_algorithm_time_taken = time.time() - FIS_algorithm_start_time
print("FIS_algorithm_time_taken",FIS_algorithm_time_taken)

Starting iteration K=1
Current interactions: [()]
Using tau threshold: 1.0
Using theta threshold: 0.8
checking the values for: [(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,), (11,), (12,)]
archipelago_tensor (13, 2, 4171)
semitruth torch.Size([4171, 1])
theta_percentile_mode True
arch_scores [-0.0061636351676689965, 0.04465719754339648, 0.09904116193157578, 0.012643305436775849, 0.3720787324466811, 0.03831873157735719, 0.0750466346536254, 0.09843771020565621, 0.03428578198949479, 0.2105832697292297, 0.23630505399855764, 0.10543624130575142, 0.0444475724083062]
theta_percentile_mode True
now having the interaction set: [(), (1,), (2,), (4,), (5,), (6,), (7,), (9,), (10,), (11,), (12,)]
Starting iteration K=2
Current interactions: [(), (1,), (2,), (4,), (5,), (6,), (7,), (9,), (10,), (11,), (12,)]
Using tau threshold: 0.5
Using theta threshold: 0.4
checking the values for: [(1, 2), (1, 4), (1, 5), (1, 6), (1, 7), (1, 9), (1, 10), (1, 11), (1, 12), (2, 4), (2, 5), (2, 

# SIAN Step 3: Train the InstaSHAP GAM

In [12]:
print("FIS_interactions")
print(FIS_interactions)

FIS_interactions
[(), (1,), (2,), (4,), (5,), (6,), (7,), (9,), (10,), (11,), (12,), (1, 2), (1, 4), (1, 6), (1, 9), (1, 11), (2, 4), (2, 6), (2, 11), (4, 5), (4, 6), (4, 7), (4, 9), (4, 11), (5, 9), (5, 11), (6, 7), (6, 11), (7, 11)]


In [ ]:
sian_training_args.model_config.FIS_interactions = FIS_interactions
sian_results = train_sian_final(dataset_obj, sian_training_args)
trained_sian = sian_results["trained_sian"]
val_tensor = sian_results["val_tensor"]


trnval_shuffle_seed 0
self.trnval_shuffle_seed 0
trnX trnY (9732, 13) (9732, 1)
Epoch 0
MSE for train and val: 2.263625412346205, 2.18300514424188
--- 1.923 seconds in epoch ---
Epoch 1
MSE for train and val: 1.2639126800035692, 1.2923571397392508
--- 1.715 seconds in epoch ---
Epoch 2
MSE for train and val: 0.6921685360556061, 0.6726624454660569
--- 1.775 seconds in epoch ---
Epoch 3
MSE for train and val: 0.6556202592924342, 0.639463607654197
--- 1.811 seconds in epoch ---
Epoch 4
MSE for train and val: 0.527062118534408, 0.5171890971150817
--- 1.931 seconds in epoch ---
Epoch 5
MSE for train and val: 0.46448562573412144, 0.4636133276827847
--- 2.188 seconds in epoch ---
Epoch 6
MSE for train and val: 0.6260337829719196, 0.608220087977765
--- 2.007 seconds in epoch ---
Epoch 7


# SIAN Step 4: Plotting Learned Shapes

In [ ]:
full_readable_labels = dataset_obj.get_full_readable_labels()
plot_all_GAM_functions(trained_sian.cpu(), val_tensor.detach().cpu().numpy(),     full_readable_labels)


In [ ]:
pass